In [1]:
import sys
!{sys.executable} -m pip install pymystem3


In [2]:
import requests
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

In [3]:
def crawler(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 100:
            return
    
        if url in visited:
            continue
        visited.add(url)
            
        try:
            #получаем контент страницы и декодируем его
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
        
        #парсинг страницы
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 10:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"    
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawler(links_tree)

In [4]:
texts = {}
visited = set()
start = 'https://tproger.ru/tag/algorithms/'

crawler([start])

In [5]:
index = {i:k for i, k in enumerate(texts)}

In [6]:
with open('/Users/Ruslan Galiev/Desktop/search/index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

In [7]:
from openpyxl import load_workbook
import pandas as pd
import os, pymorphy2

In [8]:
words_all = {}
lemma_words = {}
lemma_normal_forms = {}

In [9]:
def get_sparsed_list(word_list):
    # получить список всех словоформ
    morph, lst = pymorphy2.MorphAnalyzer(), []
    for word in word_list:
        lemma = ""
        for x in morph.parse(word)[0].lexeme:
            lemma+=x.word + " "
        lst.append(f"{word} {lemma}\n")
    return lst

In [10]:
texts_lemmatized = {}

for url in texts:
    text = texts[url]
    # оставляем только слова
    text = re.sub(r'[^a-zA-Zа-яА-Я]+', ' ', text)
   
    words = text.split(' ')
    words_all[url]=words
    
    texts_lemmatized[url]=get_sparsed_list(words_all[url])

In [11]:
with open('/Users/Ruslan Galiev/Desktop/search/words.txt', 'w') as f:
    for url in words_all:
        for i, score in enumerate(words_all[url]):
            f.write(f"{score}\n")

In [12]:
with open('/Users/Ruslan Galiev/Desktop/search/lemma_words.txt', 'w') as f:
    for url in texts_lemmatized:
        for i, score in enumerate(texts_lemmatized[url]):
            f.write(f"{score} {url}\n")

In [13]:
docs = [index[k] for k in index]

In [14]:
reverse_index = {}

for url in texts_lemmatized:
    for word in texts_lemmatized[url]:
        for word1 in word.split():
            if not word1 in reverse_index:
                 # если слово не входит в reverse_index, то добавляем его туда
                reverse_index[word1] = set([url])
            else:
                reverse_index[word1].update([url])

In [15]:
logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        query.pop(i)
        query.pop(i)
    
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [16]:
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [17]:
query = f'{x1} | {x2} & !{x3}'
print(query)

фрактале | интересуйтесь & !параметризующих


In [18]:
find(query)

{'https://tproger.ru/ad/',
 'https://tproger.ru/ad/?utm_referrer=footer',
 'https://tproger.ru/ad/jobs/?utm_referrer=feed#offer',
 'https://tproger.ru/books/obzor-knigi-kormena-i-lejzersona-algoritmy-postroenie-i-analiz/',
 'https://tproger.ru/category/devnull/',
 'https://tproger.ru/experts/how-to-learn-algorithms/',
 'https://tproger.ru/tag/sysadm/',
 'https://tproger.ru/translations/realistic-terrain-130-lines/'}

In [19]:
# 4 задание
tfs = {}

for word in reverse_index:
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs:
            tfs[word] = {}
        tfs[word][doc] = tf

In [20]:
with open('/Users/Ruslan Galiev/Desktop/search/tfs.txt', 'w') as f:
    for word in reverse_index:
        for doc in docs:
            f.write(f"{word} {doc} {tfs[word][doc]}\n")

In [21]:
pd.DataFrame(tfs).T

,https://tproger.ru/tag/algorithms/,https://tproger.ru/jobs/senior-java-developer/?utm_source=feed-jobs-widget_tag&utm_medium=tag&utm_term=Алгоритмы,https://tprg.ru/regru-footer,https://tproger.ru/tag/objective-c/,https://tproger.ru/tag/dot-net/,https://tproger.ru/search/,https://tproger.ru/tag/webassembly/,https://tproger.ru/tag/design-patterns/,https://tproger.ru/news/mehanika-tetrisa-legla-v-osnovu-algoritma-idealnogo-zapolnenija-gostinic/,https://tproger.ru/,...,https://tproger.ru/experts/getting-prepared-to-algorithms-interview/,https://tproger.ru#tagsModal,https://tproger.ru/problems/zadacha-na-sobesedovanii-provesti-prjamuju-cherez-nabor-tochek/,https://tproger.ru/category/articles/,https://tproger.ru/articles/muravej-ljengtona-zagadochnyj-kletochnyj-avtomat/,https://tproger.ru/category/digest/,https://tproger.ru/articles/why-learn-algorithms/,https://twitter.com/tproger,https://tproger.ru/tag/cpp/,https://tproger.ru/editor/
Алгоритмы,0.005274,0.002933,0.0,0.005059,0.003984,0.005792,0.005059,0.003501,0.004723,0.001944,...,0.003722,0.001944,0.001707,0.002119,0.002420,0.002016,0.005394,0.0,0.001843,0.004491
алгоритм,0.000659,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.002361,0.000000,...,0.000620,0.000000,0.000427,0.000000,0.001815,0.000000,0.003776,0.0,0.000000,0.000000
алгоритма,0.000659,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.005903,0.000000,...,0.000620,0.000000,0.000000,0.000000,0.000605,0.000000,0.000000,0.0,0.000000,0.000000
алгоритму,0.000659,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
алгоритмом,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
максимумы,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
максимумов,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
максимумам,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
максимумами,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [22]:
idfs = {}

for word in reverse_index:
    idfs[word] = math.log(len(docs) / len(reverse_index[word]))

In [23]:
with open('/Users/Ruslan Galiev/Desktop/search/idfs.txt', 'w') as f:
    for word in reverse_index:
        f.write(f"{word} {idfs[word]}\n")

In [24]:
pd.DataFrame(idfs,index=['']).T

,
Алгоритмы,0.072571
алгоритм,0.072571
алгоритма,0.072571
алгоритму,0.072571
алгоритмом,0.072571
...,...
максимумы,4.605170
максимумов,4.605170
максимумам,4.605170
максимумами,4.605170


In [25]:
tfidfs = {}

for word in tfs:
    for doc in tfs[word]:
        tfidf = tfs[word][doc] * idfs[word]
        if not word in tfidfs:
            tfidfs[word] = {}
        tfidfs[word][doc] = tfidf

In [26]:
with open('/Users/Ruslan Galiev/Desktop/search/tfidfs.txt', 'w') as f:
    for word in reverse_index:
        for doc in docs:
            f.write(f"{word} {doc} {tfidfs[word][doc]}\n")

In [27]:
pd.DataFrame(tfidfs).T

,https://tproger.ru/tag/algorithms/,https://tproger.ru/jobs/senior-java-developer/?utm_source=feed-jobs-widget_tag&utm_medium=tag&utm_term=Алгоритмы,https://tprg.ru/regru-footer,https://tproger.ru/tag/objective-c/,https://tproger.ru/tag/dot-net/,https://tproger.ru/search/,https://tproger.ru/tag/webassembly/,https://tproger.ru/tag/design-patterns/,https://tproger.ru/news/mehanika-tetrisa-legla-v-osnovu-algoritma-idealnogo-zapolnenija-gostinic/,https://tproger.ru/,...,https://tproger.ru/experts/getting-prepared-to-algorithms-interview/,https://tproger.ru#tagsModal,https://tproger.ru/problems/zadacha-na-sobesedovanii-provesti-prjamuju-cherez-nabor-tochek/,https://tproger.ru/category/articles/,https://tproger.ru/articles/muravej-ljengtona-zagadochnyj-kletochnyj-avtomat/,https://tproger.ru/category/digest/,https://tproger.ru/articles/why-learn-algorithms/,https://twitter.com/tproger,https://tproger.ru/tag/cpp/,https://tproger.ru/editor/
Алгоритмы,0.000383,0.000213,0.0,0.000367,0.000289,0.00042,0.000367,0.000254,0.000343,0.000141,...,0.000270,0.000141,0.000124,0.000154,0.000176,0.000146,0.000391,0.0,0.000134,0.000326
алгоритм,0.000048,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000171,0.000000,...,0.000045,0.000000,0.000031,0.000000,0.000132,0.000000,0.000274,0.0,0.000000,0.000000
алгоритма,0.000048,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000428,0.000000,...,0.000045,0.000000,0.000000,0.000000,0.000044,0.000000,0.000000,0.0,0.000000,0.000000
алгоритму,0.000048,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
алгоритмом,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
максимумы,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
максимумов,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
максимумам,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
максимумами,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [28]:
docs_length = {}
for row in pd.DataFrame(tfidfs).iterrows():
    sum_len = 0
    for word in row[1]:
        sum_len += word**2
    docs_length[row[0]] = math.sqrt(sum_len)

In [29]:
docs_length = {}
for row in pd.DataFrame(tfidfs).iterrows():
    sum_len = 0
    for word in row[1]:
        sum_len += word**2
    docs_length[row[0]] = math.sqrt(sum_len)

In [30]:
lemmatizer = Mystem()

Installing mystem to C:\Users\Ruslan Galiev/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


In [71]:
query_word = "курсы"
query = get_sparsed_list(query_word.lower().split())
query = " ".join([line.rstrip() for line in query])
query = query + ' ' + query_word.title()

In [72]:
tfs_query = {}
for word in query.split():
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs_query:
            tfs_query[word] = {}
        tfs_query[word][doc] = tf

In [73]:
idfs_query = {}

for word in query.split():
    idfs_query[word] = math.log(len(docs) / len(reverse_index[word]))

In [74]:
pd.DataFrame(idfs_query,index=['']).T

,
курсы,0.030459
курс,0.030459
курса,0.030459
курсу,0.030459
курсом,0.030459
курсе,0.030459
курсов,0.030459
курсам,0.030459
курсами,0.030459
курсах,0.030459


In [75]:
tfidfs_query = {}

for word in tfs_query:
    for doc in tfs_query[word]:
        tfidf = tfs_query[word][doc] * idfs_query[word]
        if not word in tfidfs_query:
            tfidfs_query[word] = {}
        tfidfs_query[word][doc] = tfidf

In [76]:
pd.DataFrame(tfidfs_query).T

,https://tproger.ru/tag/algorithms/,https://tproger.ru/jobs/senior-java-developer/?utm_source=feed-jobs-widget_tag&utm_medium=tag&utm_term=Алгоритмы,https://tprg.ru/regru-footer,https://tproger.ru/tag/objective-c/,https://tproger.ru/tag/dot-net/,https://tproger.ru/search/,https://tproger.ru/tag/webassembly/,https://tproger.ru/tag/design-patterns/,https://tproger.ru/news/mehanika-tetrisa-legla-v-osnovu-algoritma-idealnogo-zapolnenija-gostinic/,https://tproger.ru/,...,https://tproger.ru/experts/getting-prepared-to-algorithms-interview/,https://tproger.ru#tagsModal,https://tproger.ru/problems/zadacha-na-sobesedovanii-provesti-prjamuju-cherez-nabor-tochek/,https://tproger.ru/category/articles/,https://tproger.ru/articles/muravej-ljengtona-zagadochnyj-kletochnyj-avtomat/,https://tproger.ru/category/digest/,https://tproger.ru/articles/why-learn-algorithms/,https://twitter.com/tproger,https://tproger.ru/tag/cpp/,https://tproger.ru/editor/
курсы,0.00006,0.00006,0.0,0.000103,0.000121,0.000118,0.000103,0.000071,0.000072,0.000059,...,0.000038,0.000059,0.000052,0.000086,0.000037,0.000082,0.000033,0.0,0.000037,0.000091
курс,0.00002,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000036,0.000000,0.000000,...,0.000019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
курса,0.00002,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000016,0.0,0.000000,0.000000
курсу,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
курсом,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
курсе,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000020,0.000000,0.0,0.000000,0.000000
курсов,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
курсам,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
курсами,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
курсах,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [77]:
tfidfs_query_flat = pd.DataFrame(tfidfs_query).T.sum().to_dict()

In [78]:
sum_len = 1
for doc in tfidfs_query_flat:
    sum_len += tfidfs_query_flat[doc]**2
query_length = math.sqrt(sum_len)

In [79]:
cos_sim = {}

for doc in docs:
    my_list = list()
    for word in query.split(' '):
        my_list.append(((pd.Series(tfidfs_query_flat) * tfs[word][doc]).sum()) / (docs_length[doc] * query_length))
    cos_sim[doc]=max(my_list)

In [80]:
Counter(cos_sim).most_common(100)

[('https://tproger.ru/search/', 0.0036275106840872964),
 ('https://tproger.ru/tag/webassembly/', 0.002285407637366015),
 ('https://tproger.ru/video/algo-and-data-structures/?autoplay=1',
  0.0020112538460505865),
 ('https://tproger.ru/video/algorithms-for-beginners/?autoplay=1',
  0.00199361197259935),
 ('https://tproger.ru/tag/objective-c/', 0.0018623631354113181),
 ('https://tproger.ru/video/introduction-to-algorithms/',
  0.0016398155459435508),
 ('https://tproger.ru/tag/dot-net/', 0.0015877903455312164),
 ('https://tproger.ru/quiz/algorithms/', 0.0014297146161065175),
 ('https://tproger.ru/quiz/test-your-knowledge-of-ai/', 0.0013472039128958906),
 ('https://tproger.ru/tag/r-language/', 0.0013343345395976291),
 ('https://tproger.ru/tag/for-beginners/', 0.0012136175843412277),
 ('https://tproger.ru/category/articles/', 0.0012048574792618482),
 ('https://tproger.ru/tag/perl/', 0.0011392340937854543),
 ('https://tproger.ru/top/month/', 0.001114746363564396),
 ('https://tproger.ru/categ